In [1]:
import os
from PIL import Image
import vgg_model
import torch as ch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm as pbar
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import copy

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
def clean_folder(path):
    filepaths = os.listdir(path)
    do_not_work = []
    for file in filepaths:
        try:
            im = Image.open(os.path.join(path, file))
        except Exception as e:
            print(e)
            do_not_work.append(os.path.join(path, file))
    for path in do_not_work:
        os.remove(path)

In [3]:
basepath = "./car_vs_bird/car/"
for path in os.listdir(basepath):
    clean_folder(os.path.join(basepath, path))

In [4]:
# basepath = "./car_vs_bird/bird/"
# for path in os.listdir(basepath):
#     clean_folder(os.path.join(basepath, path))

In [5]:
# Before: 5177 car, 3400 bird
# After:  2123 car, 1351 bird

In [6]:
# Next up: use pretrained CIFAR-10 classifiers and finetune for 1-vs-rest concept classification (last 2 layers)

In [2]:
device = ch.device('cuda:0' if ch.cuda.is_available() else 'cpu')

In [3]:
def make_dataloaders(params):
    transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ColorJitter(.25,.25,.25),
                                          transforms.RandomRotation(2),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    
    transform_validation = transforms.Compose([transforms.ToTensor(),
                                               transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    
    trainset = torchvision.datasets.CIFAR10(root=params['path'], train=True, transform=transform_train)
    testset  = torchvision.datasets.CIFAR10(root=params['path'], train=False, transform=transform_validation)
    
    trainloader = ch.utils.data.DataLoader(trainset, batch_size=params['batch_size'], shuffle=True, num_workers=4)
    testloader  = ch.utils.data.DataLoader(testset, batch_size=params['batch_size'], shuffle=False, num_workers=4)
    return trainloader, testloader

In [4]:
def test_model(model, params, mapping, threshold=0.5):
    
    model = model.to(params['device']).eval()     
    phase = 'validation'
    logs = {'Accuracy': 0.0, 'F1-Score': 0.0}
            
    preds, gt = [], []
    # Iterate over data
    for image, label in params[phase+'_loader']:
        image = image.to(params['device'])
        # Convert labels using the given mapping
        for i, l in enumerate(label):
            gt.append(mapping[l.item()])

        with ch.no_grad():
            prediction = ch.sigmoid(model(image)).cpu()
            for pred in prediction:
                preds.append(pred >= threshold)

    logs['Accuracy'] = accuracy_score(gt, preds)
    logs['F1-Score'] = f1_score(gt, preds)
    
    return logs['Accuracy'], logs['F1-Score']

In [5]:
def finetune_model(model, params, mapping, threshold=0.3):
    
    writer = SummaryWriter('runs/' + params['description'])
    model = model.to(params['device'])
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    total_updates = params['num_epochs'] * len(params['train_loader'])
    
    criterion = ch.nn.BCEWithLogitsLoss()
    best_f1 = test_model(model, params, mapping)[1]
    best_model = copy.deepcopy(model.state_dict())
        
    for epoch in range(params['num_epochs']):
        # Each epoch has a training and validation phase
        print("Epoch : %d" % (epoch + 1))
        for phase in ['train', 'validation']:
            
            # Loss accumulator for each epoch
            logs = {'Loss': 0.0,
                    'Accuracy': 0.0,
                    'Precision': 0.0,
                    'Recall': 0.0}
            
            # Set the model to the correct phase
            model.train() if phase == 'train' else model.eval()
            
            # Iterate over data
            for image, label in params[phase+'_loader']:
                # Convert labels using the given mapping
                for i, l in enumerate(label):
                    label[i] = mapping[l.item()]
                
                image = image.to(params['device'])
                label = label.to(params['device'])

                # Zero gradient
                optimizer.zero_grad()

                with ch.set_grad_enabled(phase == 'train'):
                    
                    # Forward pass
                    prediction = model(image)
                    loss = criterion(prediction, label.unsqueeze(1).float())
                    accuracy = ch.sum((ch.sigmoid(prediction) >= threshold) == label.data).item()
                    
                    with ch.no_grad():
                        y_ = ((ch.sigmoid(prediction)).flatten().cpu().numpy() >= threshold) * 1

                    # Update log
                    logs['Loss'] += image.shape[0]*loss.detach().item()
                    logs['Accuracy'] += accuracy
                    logs['Precision'] += image.shape[0]*precision_score(label.data.cpu(), y_)
                    logs['Recall'] += image.shape[0]*recall_score(label.data.cpu(), y_)

                    # Backward pass
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
            
            # Normalize and write the data to TensorBoard
            logs['Loss'] /= len(params[phase+'_loader'].dataset)
            logs['Accuracy'] /= len(params[phase+'_loader'].dataset)
            logs['Precision'] /= len(params[phase+'_loader'].dataset)
            logs['Recall'] /= len(params[phase+'_loader'].dataset)
            
            f1_score = (2 * logs['Precision'] * logs['Recall']) / (logs['Precision'] + logs['Recall'])
            writer.add_scalars('Loss', {phase: logs['Loss']}, epoch)
            writer.add_scalars('Accuracy', {phase: logs['Accuracy']}, epoch)
            writer.add_scalars('F-1', {phase: f1_score}, epoch)
            
            print("%s loss: %.3f , accuracy: %.3f, F-1: %.3f" % (phase, logs['Loss'], logs['Accuracy'], f1_score))

            # Save the best weights
            if phase == 'validation' and f1_score > best_f1:
                best_f1 = f1_score
                best_model = copy.deepcopy(model.state_dict())
                 
        # Write best weights to disk
        if epoch % params['check_point'] == 0 or epoch == params['num_epochs']-1:
            ch.save(best_model, params['description'] + '.pt')
    
    final_accuracy = test_model(model, params, mapping)[0]
    writer.add_text('Final_Accuracy', str(final_accuracy), 0)
    writer.close()

In [6]:
def make_folder_loaders(params):
    transform_train = transforms.Compose([transforms.Resize((32, 32)),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ColorJitter(.25,.25,.25),
                                          transforms.RandomRotation(2),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    
    transform_validation = transforms.Compose([transforms.Resize((32, 32)),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    
    trainset = torchvision.datasets.ImageFolder(root=os.path.join(params['path'], "train"), transform=transform_train)
    testset  = torchvision.datasets.ImageFolder(root=os.path.join(params['path'], "val"),   transform=transform_validation)
    
    trainloader = ch.utils.data.DataLoader(trainset, batch_size=params['batch_size'], shuffle=True, num_workers=4)
    testloader  = ch.utils.data.DataLoader(testset, batch_size=params['batch_size'], shuffle=False, num_workers=4)
    return trainloader, testloader

In [7]:
def finetune_into_binary(m):
    # Freeze feature extraction layers
    for n, l in m.named_parameters():
        if "features." in n:
            l.requires_grad  = False
    # Swap out final classification layer
    m.classifier[6] = ch.nn.Linear(m.classifier[6].weight.shape[1], 1)
    return m

### Bird Concept Classifiers

In [12]:
basepath = "./car_vs_bird/bird/train"
bird_classifiers = {x:None for x in os.listdir(basepath)}
print(bird_classifiers)

{'feather': None, 'feet': None, 'beak': None, 'eyes': None, 'wings': None}


In [14]:
data_params = {'path': '/p/adversarialml/as9rw/dlvr_project/car_vs_bird/bird/', 'batch_size': 128}
train_loader, validation_loader = make_folder_loaders(data_params)
train_params = {'description': 'Bird-Concepts',
                'num_epochs': 10,
                'check_point': 5, 'device': device,
                'train_loader': train_loader, 'validation_loader': validation_loader}

In [15]:
print(train_loader.dataset.classes)
print(train_loader.dataset.class_to_idx)
print(validation_loader.dataset.classes)
print(validation_loader.dataset.class_to_idx)

['beak', 'eyes', 'feather', 'feet', 'wings']
{'beak': 0, 'eyes': 1, 'feather': 2, 'feet': 3, 'wings': 4}
['beak', 'eyes', 'feather', 'feet', 'wings']
{'beak': 0, 'eyes': 1, 'feather': 2, 'feet': 3, 'wings': 4}


In [16]:
bird_classifiers['beak'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))

In [17]:
beak_mapping = {0:1, 1:0, 2:0, 3:0, 4:0}
finetune_model(bird_classifiers['beak'], train_params, beak_mapping)

Epoch : 1
train loss: 0.662 , accuracy: 36.755, F-1: 0.445


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


validation loss: 0.620 , accuracy: 32.737, F-1: 0.274
Epoch : 2
train loss: 0.623 , accuracy: 40.076, F-1: 0.447
validation loss: 0.595 , accuracy: 36.290, F-1: 0.273
Epoch : 3
train loss: 0.602 , accuracy: 44.632, F-1: 0.446
validation loss: 0.572 , accuracy: 45.668, F-1: 0.262
Epoch : 4
train loss: 0.596 , accuracy: 49.156, F-1: 0.444
validation loss: 0.561 , accuracy: 54.912, F-1: 0.257
Epoch : 5
train loss: 0.589 , accuracy: 57.083, F-1: 0.444
validation loss: 0.551 , accuracy: 65.484, F-1: 0.252
Epoch : 6
train loss: 0.587 , accuracy: 61.983, F-1: 0.427
validation loss: 0.548 , accuracy: 70.235, F-1: 0.238
Epoch : 7
train loss: 0.584 , accuracy: 64.463, F-1: 0.439
validation loss: 0.548 , accuracy: 71.350, F-1: 0.238
Epoch : 8
train loss: 0.581 , accuracy: 64.947, F-1: 0.456


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


validation loss: 0.544 , accuracy: 70.977, F-1: 0.240
Epoch : 9
train loss: 0.583 , accuracy: 64.844, F-1: 0.452
validation loss: 0.545 , accuracy: 67.765, F-1: 0.245
Epoch : 10
train loss: 0.578 , accuracy: 61.785, F-1: 0.476
validation loss: 0.542 , accuracy: 70.092, F-1: 0.237


In [18]:
for i in range(11):
    acc = test_model(bird_classifiers['beak'], train_params, beak_mapping, i / 10)
    print("Threshold %.2f : Acc %.3f, F-1 %.3f" % (i/10, acc[0], acc[1]))

Threshold 0.00 : Acc 0.256, F-1 0.407
Threshold 0.10 : Acc 0.256, F-1 0.407
Threshold 0.20 : Acc 0.300, F-1 0.420
Threshold 0.30 : Acc 0.613, F-1 0.485
Threshold 0.40 : Acc 0.744, F-1 0.000
Threshold 0.50 : Acc 0.744, F-1 0.000
Threshold 0.60 : Acc 0.744, F-1 0.000
Threshold 0.70 : Acc 0.744, F-1 0.000
Threshold 0.80 : Acc 0.744, F-1 0.000
Threshold 0.90 : Acc 0.744, F-1 0.000
Threshold 1.00 : Acc 0.744, F-1 0.000


In [19]:
bird_classifiers['eyes'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
eyes_mapping = {0:0, 1:1, 2:0, 3:0, 4:0}
finetune_model(bird_classifiers['eyes'], train_params, eyes_mapping)

Epoch : 1
train loss: 0.624 , accuracy: 19.035, F-1: 0.220
validation loss: 0.514 , accuracy: 19.788, F-1: 0.171
Epoch : 2
train loss: 0.487 , accuracy: 52.699, F-1: 0.192
validation loss: 0.451 , accuracy: 68.244, F-1: 0.159
Epoch : 3
train loss: 0.440 , accuracy: 85.583, F-1: 0.210
validation loss: 0.411 , accuracy: 103.088, F-1: 0.017
Epoch : 4
train loss: 0.413 , accuracy: 107.144, F-1: 0.068
validation loss: 0.400 , accuracy: 104.051, F-1: nan
Epoch : 5


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in double_scalars


train loss: 0.405 , accuracy: 108.925, F-1: 0.021
validation loss: 0.387 , accuracy: 104.267, F-1: nan
Epoch : 6
train loss: 0.398 , accuracy: 110.535, F-1: 0.013
validation loss: 0.384 , accuracy: 104.267, F-1: nan
Epoch : 7
train loss: 0.392 , accuracy: 110.194, F-1: 0.016
validation loss: 0.379 , accuracy: 104.267, F-1: nan
Epoch : 8
train loss: 0.394 , accuracy: 110.258, F-1: nan
validation loss: 0.375 , accuracy: 104.267, F-1: nan
Epoch : 9
train loss: 0.396 , accuracy: 110.503, F-1: 0.018
validation loss: 0.373 , accuracy: 104.267, F-1: nan
Epoch : 10
train loss: 0.384 , accuracy: 110.265, F-1: 0.016
validation loss: 0.369 , accuracy: 104.267, F-1: nan


In [20]:
for i in range(11):
    acc = test_model(bird_classifiers['eyes'], train_params, eyes_mapping, i / 10)
    print("Threshold %.2f : Acc %.3f, F-1 %.3f" % (i/10, acc[0], acc[1]))

Threshold 0.00 : Acc 0.115, F-1 0.207
Threshold 0.10 : Acc 0.233, F-1 0.190
Threshold 0.20 : Acc 0.841, F-1 0.127
Threshold 0.30 : Acc 0.885, F-1 0.000
Threshold 0.40 : Acc 0.885, F-1 0.000
Threshold 0.50 : Acc 0.885, F-1 0.000
Threshold 0.60 : Acc 0.885, F-1 0.000
Threshold 0.70 : Acc 0.885, F-1 0.000
Threshold 0.80 : Acc 0.885, F-1 0.000
Threshold 0.90 : Acc 0.885, F-1 0.000
Threshold 1.00 : Acc 0.885, F-1 0.000


In [21]:
bird_classifiers['feather'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
feather_mapping = {0:0, 1:0, 2:1, 3:0, 4:0}
finetune_model(bird_classifiers['feather'], train_params, feather_mapping)

Epoch : 1
train loss: 0.594 , accuracy: 21.633, F-1: 0.243
validation loss: 0.502 , accuracy: 20.995, F-1: 0.181
Epoch : 2
train loss: 0.485 , accuracy: 55.654, F-1: 0.248
validation loss: 0.438 , accuracy: 92.677, F-1: 0.027
Epoch : 3
train loss: 0.447 , accuracy: 97.543, F-1: 0.087
validation loss: 0.411 , accuracy: 102.498, F-1: nan
Epoch : 4


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in double_scalars


train loss: 0.429 , accuracy: 107.683, F-1: nan
validation loss: 0.400 , accuracy: 102.498, F-1: nan
Epoch : 5
train loss: 0.419 , accuracy: 108.590, F-1: nan
validation loss: 0.397 , accuracy: 102.498, F-1: nan
Epoch : 6
train loss: 0.419 , accuracy: 109.336, F-1: nan
validation loss: 0.396 , accuracy: 102.498, F-1: nan
Epoch : 7
train loss: 0.425 , accuracy: 109.231, F-1: nan
validation loss: 0.394 , accuracy: 102.498, F-1: nan
Epoch : 8
train loss: 0.415 , accuracy: 109.504, F-1: nan
validation loss: 0.392 , accuracy: 102.498, F-1: nan
Epoch : 9
train loss: 0.415 , accuracy: 109.604, F-1: nan
validation loss: 0.391 , accuracy: 102.498, F-1: nan
Epoch : 10
train loss: 0.416 , accuracy: 109.336, F-1: nan
validation loss: 0.391 , accuracy: 102.498, F-1: nan


In [22]:
for i in range(11):
    acc = test_model(bird_classifiers['feather'], train_params, feather_mapping, i / 10)
    print("Threshold %.2f : Acc %.3f, F-1 %.3f" % (i/10, acc[0], acc[1]))

Threshold 0.00 : Acc 0.129, F-1 0.229
Threshold 0.10 : Acc 0.182, F-1 0.227
Threshold 0.20 : Acc 0.717, F-1 0.163
Threshold 0.30 : Acc 0.871, F-1 0.000
Threshold 0.40 : Acc 0.871, F-1 0.000
Threshold 0.50 : Acc 0.871, F-1 0.000
Threshold 0.60 : Acc 0.871, F-1 0.000
Threshold 0.70 : Acc 0.871, F-1 0.000
Threshold 0.80 : Acc 0.871, F-1 0.000
Threshold 0.90 : Acc 0.871, F-1 0.000
Threshold 1.00 : Acc 0.871, F-1 0.000


In [23]:
bird_classifiers['feet'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
feet_mapping = {0:0, 1:0, 2:0, 3:1, 4:0}
finetune_model(bird_classifiers['feet'], train_params, feet_mapping)

Epoch : 1
train loss: 0.648 , accuracy: 24.633, F-1: 0.319
validation loss: 0.561 , accuracy: 21.940, F-1: 0.265
Epoch : 2
train loss: 0.552 , accuracy: 35.755, F-1: 0.321
validation loss: 0.523 , accuracy: 28.848, F-1: 0.267
Epoch : 3
train loss: 0.528 , accuracy: 54.104, F-1: 0.315
validation loss: 0.489 , accuracy: 74.465, F-1: 0.199
Epoch : 4
train loss: 0.511 , accuracy: 79.126, F-1: 0.244
validation loss: 0.478 , accuracy: 89.581, F-1: 0.106
Epoch : 5
train loss: 0.511 , accuracy: 85.882, F-1: 0.183
validation loss: 0.470 , accuracy: 93.323, F-1: 0.079
Epoch : 6
train loss: 0.504 , accuracy: 94.627, F-1: 0.121
validation loss: 0.465 , accuracy: 95.567, F-1: 0.029
Epoch : 7
train loss: 0.502 , accuracy: 96.677, F-1: 0.070
validation loss: 0.463 , accuracy: 96.263, F-1: 0.029
Epoch : 8
train loss: 0.503 , accuracy: 97.300, F-1: 0.099
validation loss: 0.460 , accuracy: 96.645, F-1: 0.015
Epoch : 9
train loss: 0.496 , accuracy: 98.767, F-1: 0.056
validation loss: 0.457 , accuracy: 97

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in double_scalars


train loss: 0.491 , accuracy: 101.465, F-1: 0.010
validation loss: 0.454 , accuracy: 97.189, F-1: nan


In [24]:
for i in range(11):
    acc = test_model(bird_classifiers['feet'], train_params, feet_mapping, i / 10)
    print("Threshold %.2f : Acc %.3f, F-1 %.3f" % (i/10, acc[0], acc[1]))

Threshold 0.00 : Acc 0.171, F-1 0.291
Threshold 0.10 : Acc 0.191, F-1 0.294
Threshold 0.20 : Acc 0.581, F-1 0.278
Threshold 0.30 : Acc 0.829, F-1 0.000
Threshold 0.40 : Acc 0.829, F-1 0.000
Threshold 0.50 : Acc 0.829, F-1 0.000
Threshold 0.60 : Acc 0.829, F-1 0.000
Threshold 0.70 : Acc 0.829, F-1 0.000
Threshold 0.80 : Acc 0.829, F-1 0.000
Threshold 0.90 : Acc 0.829, F-1 0.000
Threshold 1.00 : Acc 0.829, F-1 0.000


In [25]:
bird_classifiers['wings'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
wings_mapping = {0:0, 1:0, 2:0, 3:0, 4:1}
finetune_model(bird_classifiers['wings'], train_params, wings_mapping)

Epoch : 1
train loss: 0.694 , accuracy: 32.551, F-1: 0.403
validation loss: 0.653 , accuracy: 33.189, F-1: 0.365
Epoch : 2
train loss: 0.626 , accuracy: 34.577, F-1: 0.401
validation loss: 0.641 , accuracy: 34.691, F-1: 0.362
Epoch : 3
train loss: 0.608 , accuracy: 41.639, F-1: 0.380
validation loss: 0.636 , accuracy: 47.235, F-1: 0.325
Epoch : 4
train loss: 0.587 , accuracy: 55.869, F-1: 0.338
validation loss: 0.633 , accuracy: 57.742, F-1: 0.290
Epoch : 5
train loss: 0.586 , accuracy: 60.776, F-1: 0.314
validation loss: 0.634 , accuracy: 74.618, F-1: 0.205
Epoch : 6
train loss: 0.582 , accuracy: 73.295, F-1: 0.284
validation loss: 0.633 , accuracy: 75.765, F-1: 0.218
Epoch : 7
train loss: 0.566 , accuracy: 74.092, F-1: 0.305
validation loss: 0.622 , accuracy: 67.171, F-1: 0.278
Epoch : 8
train loss: 0.570 , accuracy: 65.551, F-1: 0.369
validation loss: 0.624 , accuracy: 78.318, F-1: 0.236
Epoch : 9
train loss: 0.563 , accuracy: 72.214, F-1: 0.335
validation loss: 0.632 , accuracy: 81

In [26]:
for i in range(11):
    acc = test_model(bird_classifiers['wings'], train_params, wings_mapping, i / 10)
    print("Threshold %.2f : Acc %.3f, F-1 %.3f" % (i/10, acc[0], acc[1]))

Threshold 0.00 : Acc 0.329, F-1 0.496
Threshold 0.10 : Acc 0.329, F-1 0.496
Threshold 0.20 : Acc 0.376, F-1 0.490
Threshold 0.30 : Acc 0.682, F-1 0.425
Threshold 0.40 : Acc 0.671, F-1 0.000
Threshold 0.50 : Acc 0.671, F-1 0.000
Threshold 0.60 : Acc 0.671, F-1 0.000
Threshold 0.70 : Acc 0.671, F-1 0.000
Threshold 0.80 : Acc 0.671, F-1 0.000
Threshold 0.90 : Acc 0.671, F-1 0.000
Threshold 1.00 : Acc 0.671, F-1 0.000


In [27]:
for k, v in bird_classifiers.items():
    ch.save(v, "./concept_classifiers/bird/" + k + '.pt')

### Car Concept Classifiers

In [8]:
data_params = {'path': '/p/adversarialml/as9rw/dlvr_project/car_vs_bird/car/', 'batch_size': 128}
train_loader, validation_loader = make_folder_loaders(data_params)
train_params = {'description': 'Car-Concepts',
                'num_epochs': 10,
                'check_point': 5, 'device': device,
                'train_loader': train_loader, 'validation_loader': validation_loader}

In [9]:
print(train_loader.dataset.classes)
print(train_loader.dataset.class_to_idx)
print(validation_loader.dataset.classes)
print(validation_loader.dataset.class_to_idx)

['bumper', 'door', 'headlight', 'side-view-mirror', 'tail-light', 'tire', 'window']
{'bumper': 0, 'door': 1, 'headlight': 2, 'side-view-mirror': 3, 'tail-light': 4, 'tire': 5, 'window': 6}
['bumper', 'door', 'headlight', 'side-view-mirror', 'tail-light', 'tire', 'window']
{'bumper': 0, 'door': 1, 'headlight': 2, 'side-view-mirror': 3, 'tail-light': 4, 'tire': 5, 'window': 6}


In [13]:
basepath = "./car_vs_bird/car/train"
car_classifiers = {x:None for x in os.listdir(basepath)}
print(car_classifiers)

{'tail-light': None, 'side-view-mirror': None, 'bumper': None, 'tire': None, 'headlight': None, 'window': None, 'door': None}


In [31]:
car_classifiers['bumper'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
bumper_mapping = {0:1, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}
finetune_model(car_classifiers['bumper'], train_params, bumper_mapping)

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch : 1


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.556 , accuracy: 24.166, F-1: 0.181


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.330 , accuracy: 112.248, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


validation loss: 0.335 , accuracy: 113.316, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [33]:
car_classifiers['door'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
door_mapping = {0:0, 1:1, 2:0, 3:0, 4:0, 5:0, 6:0}
finetune_model(car_classifiers['door'], train_params, door_mapping)

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch : 1


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.522 , accuracy: 31.061, F-1: 0.206


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.324 , accuracy: 111.380, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


validation loss: 0.322 , accuracy: 112.506, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [14]:
car_classifiers['headlight'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
headlight_mapping = {0:0, 1:0, 2:1, 3:0, 4:0, 5:0, 6:0}
finetune_model(car_classifiers['headlight'], train_params, headlight_mapping)

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch : 1


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.639 , accuracy: 25.314, F-1: 0.318


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.490 , accuracy: 99.990, F-1: 0.055


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


validation loss: 0.494 , accuracy: 102.380, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [15]:
car_classifiers['side-view-mirror'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
svm_mapping = {0:0, 1:0, 2:0, 3:1, 4:0, 5:0, 6:0}
finetune_model(car_classifiers['side-view-mirror'], train_params, svm_mapping)

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch : 1


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.542 , accuracy: 24.701, F-1: 0.190


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.333 , accuracy: 112.256, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


validation loss: 0.349 , accuracy: 112.709, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [16]:
car_classifiers['tail-light'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
taillight_mapping = {0:0, 1:0, 2:0, 3:0, 4:1, 5:0, 6:0}
finetune_model(car_classifiers['tail-light'], train_params, taillight_mapping)

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch : 1


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.558 , accuracy: 22.050, F-1: 0.184


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.319 , accuracy: 112.902, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


validation loss: 0.320 , accuracy: 113.722, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [17]:
car_classifiers['tire'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
tire_mapping = {0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:0}
finetune_model(car_classifiers['tire'], train_params, tire_mapping)

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch : 1


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.641 , accuracy: 29.782, F-1: 0.337


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.470 , accuracy: 83.165, F-1: 0.487


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


validation loss: 0.470 , accuracy: 97.525, F-1: 0.230


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [18]:
car_classifiers['window'] = finetune_into_binary(vgg_model.vgg19_bn(pretrained=True))
window_mapping = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:1}
finetune_model(car_classifiers['window'], train_params, window_mapping)

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch : 1


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.584 , accuracy: 28.830, F-1: 0.303


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


...

/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train loss: 0.466 , accuracy: 100.590, F-1: 0.080


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


validation loss: 0.458 , accuracy: 104.835, F-1: nan


/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/u/as9rw/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [20]:
for k, v in car_classifiers.items():
    ch.save(v, "./concept_classifiers/car/" + k + '.pt')

In [68]:
import torchvision
import torchvision.transforms as transforms

# Construct artificial CIFAR10 car/bird data
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ColorJitter(.25,.25,.25),
                                          transforms.RandomRotation(2),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    
transform_validation = transforms.Compose([transforms.ToTensor(),
                                          transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = ch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_validation)
testloader = ch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [69]:
X_train, Y_train, X_val, Y_val = [], [], [], []
for x,y in trainloader:
    X_train.append(x)
    Y_train.append(y)
X_train = ch.cat(X_train, 0)
Y_train = ch.cat(Y_train, 0)

In [70]:
# Filter out relevant data
car_indices = (Y_train == 1)
bird_indices = (Y_train == 2)

print(X_train[car_indices].shape, X_train[bird_indices].shape)
X_train_ = ch.cat([X_train[car_indices], X_train[bird_indices]], 0)
Y_train_ = ch.cat([Y_train[car_indices], Y_train[bird_indices]], 0)

torch.Size([5000, 3, 32, 32]) torch.Size([5000, 3, 32, 32])


In [71]:
for x,y in testloader:
    X_val.append(x)
    Y_val.append(y)
X_val = ch.cat(X_val, 0)
Y_val = ch.cat(Y_val, 0)

In [72]:
# Filter out relevant data
car_indices = (Y_val == 1)
bird_indices = (Y_val == 2)

print(X_val[car_indices].shape, X_val[bird_indices].shape)
X_val_ = ch.cat([X_val[car_indices], X_val[bird_indices]], 0)
Y_val_ = ch.cat([Y_val[car_indices], Y_val[bird_indices]], 0)

torch.Size([1000, 3, 32, 32]) torch.Size([1000, 3, 32, 32])


In [76]:
Y_val_ = (Y_val_ == 2) * 1
Y_train_ = (Y_train_ == 2) * 1

In [46]:
from tqdm import tqdm

X_actual_train = []
for x in tqdm(X_train_):
    features = []
    for k, v in car_classifiers.items():
        if v is None:
            continue
        score = v(x.unsqueeze(0).cuda())
        features.append(score.item())
    X_actual_train.append(features)

100%|██████████| 10000/10000 [03:13<00:00, 51.67it/s]


In [47]:
X_actual_val = []
for x in tqdm(X_val_):
    features = []
    for k, v in car_classifiers.items():
        if v is None:
            continue
        score = v(x.unsqueeze(0).cuda())
        features.append(score.item())
    X_actual_val.append(features)

100%|██████████| 2000/2000 [00:38<00:00, 51.67it/s]


In [79]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_actual_train, Y_train_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [80]:
clf.score(X_actual_val, Y_val_)

0.9765

In [92]:
clf.score(X_actual_val, Y_val_)

0.9765